# Clustering: Strategic New Fitness Centers in Los Angeles

### IBM Applied Data Science Capstone Project

# Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Introduction: Business Problem</a></li>
        <li><a href="#ref2">Data</a></li>
        <li><a href="#ref3">Methodology</a><ul>
            <li><a href="#ref4">Data Gathering</a></li>
            <li><a href="#ref5">Geocoding LA Neighborhoods</a></li>
            <li><a href="#ref6">Data Visualization: Mapping Los Angeles</a></li>
            <li><a href="#ref7">Exploring Neighborhood Venues With Foursquare API</a></li>
            <li><a href="#ref8">Clustering Neighborhoods by Gym & Fitness Center Data</a></li>
        </ul>
        </li>
        <li><a href="#ref9">Results</a></li>
        <li><a href="#ref10">Discussion</a></li>
        <li><a href="#ref11">Conclusion</a></li>
    </ol>
</div>

<a id="ref1"></a> 
# Introduction: Business Problem

The goal is to minimize travel for all Los Angeles residents in need of access to a local gym while promoting health across the city. The objective of this project is to determine optimal locations for a new gym or fitness center within Los Angeles. Specifically, the end result is to compile a list of Los Angeles neighborhoods that have no gyms or fitness centers within 3000 meters from their center points. 

<a id="ref2"></a> 
# Data

The following data resources will be used for this project:


+ The official list of neighborhoods in Los Angeles. 
    + Description: Names of each Los Angeles neighborhood are listed. The scope of this project does not extend to the greater Los Angeles area and is confined to the official neighborhood boundaries under Los Angeles. 
    + Data Source: geohub.lacity.org
+ Coordinates for each Los Angeles neighborhood.
    + Description: Latitude and longitude coordinates for the center of each Los Angeles neighborhood. These coordinates will be used to plot circle markers over a map of Los Angeles and to explore each of them.
    + Source: geopy.geocoder package
+ Venue Data in proximity of each Los Angeles neighborhood.
    + Description: Using Foursquare API a search query will be done against the coordinates for each Los Angeles neighborhood and venue data that is within a radius of 3000 meters of each neighborhood will be generated. This venue data will then be filtered to account for only data related to gyms and fitness centers.
    + Data source: Foursquare API

<a id="ref3"></a> 
# Methodology

In [1]:
%%capture
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

from bs4 import BeautifulSoup # library to parse HTML and XML documents

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

<a id="ref4"></a> 
### Data Gathering

Getting the link to the LA neighborhood data uploaded in a github repository.

In [2]:
url = "https://github.com/c2barreto/Coursera_Captstone/blob/main/LA_Times_Neighborhood_Boundaries.csv" 

Passing the URL to a Pandas dataframe.

In [3]:
df_Neighbs = pd.read_html(url)

Pass the table of interest from the url into a Pandas data frame.

In [4]:
df_LA = df_Neighbs[0]

Dropping unnecessary columns.

In [5]:
df_LA.drop(["Unnamed: 0", "OBJECTID"], inplace=True, axis=1) 

Renaming columns

In [6]:
df_LA.rename(columns = {'name':'Neighborhood'}, inplace = True)

In [7]:
df_LA.head()

,Neighborhood
0,Adams-Normandie
1,Arleta
2,Arlington Heights
3,Atwater Village
4,Baldwin Hills/Crenshaw


In [8]:
df_LA.shape

(114, 1)

<a id="ref5"></a> 
### Geocoding LA Neighborhoods

Defining a function to retrieve each LA neighborhood's coordinates

In [10]:
def get_latlng(LAneighborhood):
    
    # initialize a variable to None
    lat_lng_coords = None
    
    # loop until all neighborhood coordinates are retrieved using geocoder package
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Los Angeles, California'.format(LAneighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

Calling the defined function to get the neighborhood coordinates and then storing the values into a list.

In [11]:
coordList = [ get_latlng(LAneighborhood) for LAneighborhood in df_LA["Neighborhood"].tolist() ]

Creating a temporary dataframe to populate the coordinates.

In [12]:
df_temp = pd.DataFrame(coordList, columns=['Latitude', 'Longitude'])

Merging the temporary dataframe columns with the df_LA.

In [13]:
df_LA['Latitude'] = df_temp['Latitude']
df_LA['Longitude'] = df_temp['Longitude']

In [14]:
df_LA.head()

,Neighborhood,Latitude,Longitude
0,Adams-Normandie,33.901212,-118.299321
1,Arleta,34.249050,-118.433490
2,Arlington Heights,34.039890,-118.325160
3,Atwater Village,34.119700,-118.258870
4,Baldwin Hills/Crenshaw,34.010094,-118.337336


In [15]:
df_LA.to_csv("df_LA3.csv", index=False)

<a id="ref6"></a> 
### Data Visualization: Mapping Los Angeles

Getting the coordinates of Los Angeles.

In [16]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.242766.


Creating a map of Los Angeles by passing the latitude and longitude values for each neighborhood.

In [17]:
map_LA = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for lat, lng, neighborhood in zip(df_LA['Latitude'], df_LA['Longitude'], df_LA['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_LA)  
    
map_LA

In [18]:
# saving the map as HTML file
map_LA.save('map_LA3.html')

<a id="ref7"></a> 
### Exploring Neighborhood Venues With Foursquare API

Defining foursquare credentials and its version in order to access its database.

In [19]:
CLIENT_ID = 'H0ZIZZAEGBFDEGUJYC0XZKX4Z2ROHFX53WN0V5SENBVMDBVU' # my Foursquare ID
CLIENT_SECRET = 'D1HOVQ0X0UQV1JHXSZ0TIC0HFODDSELZH2WTWLJ0U5GQ5E1A' # my Foursquare Secret
VERSION = '20201105' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H0ZIZZAEGBFDEGUJYC0XZKX4Z2ROHFX53WN0V5SENBVMDBVU
CLIENT_SECRET:D1HOVQ0X0UQV1JHXSZ0TIC0HFODDSELZH2WTWLJ0U5GQ5E1A


Defining a function to get top venues of each neighborhood within a 500 meter radius.

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # creating the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # making the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # returning only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Running the above function to query each neighborhood through the foursquare API while passing the venue data to a new dataframe called losangeles_venues.

In [21]:
%%capture
losangeles_venues = getNearbyVenues(names=df_LA['Neighborhood'],
                                   latitudes=df_LA['Latitude'],
                                   longitudes=df_LA['Longitude']
                                  )

In [22]:
print(losangeles_venues.shape)
losangeles_venues.head()

(9668, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams-Normandie,33.901212,-118.299321,Popeyes Louisiana Kitchen,33.901664,-118.295854,Fried Chicken Joint
1,Adams-Normandie,33.901212,-118.299321,Starbucks,33.901643,-118.300039,Coffee Shop
2,Adams-Normandie,33.901212,-118.299321,Spoon House,33.889974,-118.304900,Italian Restaurant
3,Adams-Normandie,33.901212,-118.299321,Crazy Rock'n Sushi,33.890100,-118.303242,Sushi Restaurant
4,Adams-Normandie,33.901212,-118.299321,State Brewing Co.,33.910972,-118.297760,Brewery


Checking how many venues were returned for each neighborhood.

In [23]:
losangeles_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adams-Normandie,100,100,100,100,100,100
Arleta,60,60,60,60,60,60
Arlington Heights,100,100,100,100,100,100
Atwater Village,100,100,100,100,100,100
Baldwin Hills/Crenshaw,76,76,76,76,76,76
Bel-Air,66,66,66,66,66,66
Beverly Crest,43,43,43,43,43,43
Beverly Grove,23,23,23,23,23,23
Beverlywood,100,100,100,100,100,100


Checking how many unique categories can be curated from all the returned venues.

In [24]:
print('There are {} uniques categories.'.format(len(losangeles_venues['Venue Category'].unique())))

There are 412 uniques categories.


Creating a one hot encoding technique that turns each unique venue category into a column and resets the index to be by neighborhood.

In [26]:
# one hot encoding
losangeles_onehot = pd.get_dummies(losangeles_venues[['Venue Category']], prefix="", prefix_sep="")

# adding neighborhood column back to dataframe
losangeles_onehot['Neighborhood'] = losangeles_venues['Neighborhood'] 

# defining a list of column names
cols = losangeles_onehot.columns.tolist()
cols

# moving neighborhood column to the first column
cols.insert(0, cols.pop(cols.index('Neighborhood')))
cols

#applying .reindex() function to reorder
losangeles_onehot = losangeles_onehot.reindex(columns= cols)

#checking result
losangeles_onehot.head()


,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Casino,Cave,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Gym,College Rec Center,College Residence Hall,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dam,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Marine Terminal,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Observatory,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Sh

In [27]:
#viewing the new dataframe size
losangeles_onehot.shape

(9668, 412)

Grouping the rows by neighborhood and taking the mean for the frequency of occurrence for each venue category.

In [28]:
losangeles_grouped = losangeles_onehot.groupby('Neighborhood').mean().reset_index()
losangeles_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Casino,Cave,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Gym,College Rec Center,College Residence Hall,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dam,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Marine Terminal,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Observatory,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Sh

In [29]:
#confirming new size
losangeles_grouped.shape

(114, 412)

<a id="ref8"></a> 
### Clustering Neighborhoods by Gym & Fitness Center Data

Getting a count of neighborhoods with at least 1 occurance of a gym or fitness center near them.

In [30]:
len(losangeles_grouped[losangeles_grouped["Gym / Fitness Center"] > 0])

46

Creating a new data frame for the LA neighborhood data on Gyms and Fitness Centers.

In [31]:
LA_gyms = losangeles_grouped[["Neighborhood","Gym / Fitness Center"]]

In [32]:
LA_gyms.head()

,Neighborhood,Gym / Fitness Center
0,Adams-Normandie,0.00
1,Arleta,0.00
2,Arlington Heights,0.00
3,Atwater Village,0.02
4,Baldwin Hills/Crenshaw,0.00


Getting the K-Means Clustering technique setup.

In [33]:
# setting number of clusters
kclusters = 5

losangeles_grouped_clustering = LA_gyms.drop('Neighborhood', 1)

# running k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(losangeles_grouped_clustering)

# checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 4, 2, 1, 1, 1, 1, 0, 1, 1, 3, 1, 3,
       3, 0, 0, 1, 3, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 3, 1, 1, 1, 3, 1, 0, 1, 0, 2, 1, 0, 1, 0, 0, 0, 1, 4, 3, 1, 2,
       1, 1, 1, 3, 1, 1, 1, 0, 0, 1, 2, 1], dtype=int32)

In [34]:
LA_merged = LA_gyms.copy()
# adding cluster labels column 
LA_merged["Cluster Labels"] = kmeans.labels_

LA_merged.head() # check the last column

,Neighborhood,Gym / Fitness Center,Cluster Labels
0,Adams-Normandie,0.00,1
1,Arleta,0.00,1
2,Arlington Heights,0.00,1
3,Atwater Village,0.02,3
4,Baldwin Hills/Crenshaw,0.00,1


Merging df_LA with LA_merged to add latitude/longitude for each neighborhood.

In [35]:
LA_merged = LA_merged.join(df_LA.set_index("Neighborhood"), on="Neighborhood")

print(LA_merged.shape)
LA_merged.head() # check the last columns

(114, 5)


,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
0,Adams-Normandie,0.00,1,33.901212,-118.299321
1,Arleta,0.00,1,34.249050,-118.433490
2,Arlington Heights,0.00,1,34.039890,-118.325160
3,Atwater Village,0.02,3,34.119700,-118.258870
4,Baldwin Hills/Crenshaw,0.00,1,34.010094,-118.337336


Visualizing the resulting clusters.

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LA_merged['Latitude'], LA_merged['Longitude'], LA_merged['Neighborhood'], LA_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
map_clusters.save('map_clusters.html')

<a id="ref9"></a> 
# Results

### Cluster 0 (Red)

In [38]:
cluster0 = LA_merged.loc[LA_merged['Cluster Labels'] == 0]
cluster0

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
16,Chatsworth,0.010000,0,34.257250,-118.591010
23,Downtown,0.010000,0,34.048340,-118.255640
24,Eagle Rock,0.010000,0,34.139270,-118.210870
30,Encino,0.010000,0,34.165380,-118.527110
32,Fairfax,0.010000,0,34.076100,-118.361220
41,Harbor City,0.010000,0,33.797810,-118.302480
44,Harvard Park,0.010000,0,34.144400,-118.252970
45,Highland Park,0.010000,0,34.109820,-118.192320
47,Hollywood,0.010000,0,34.101560,-118.337080
48,Hollywood Hills,0.010000,0,34.105240,-118.320650


In [39]:
cluster0.shape

(28, 5)

Cluster 0 groups together 28 neighborhoods with a low concentration of fitness centers in proximity. The venue data for these neighborhoods generated at least 1 occurrence of a fitness center within a 2000 meter radius from their center points.

### Cluster 1 (Purple)

In [40]:
cluster1 = LA_merged.loc[LA_merged['Cluster Labels'] == 1]
cluster1

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
0,Adams-Normandie,0.0,1,33.901212,-118.299321
1,Arleta,0.0,1,34.249050,-118.433490
2,Arlington Heights,0.0,1,34.039890,-118.325160
4,Baldwin Hills/Crenshaw,0.0,1,34.010094,-118.337336
5,Bel-Air,0.0,1,34.083610,-118.434830
7,Beverly Grove,0.0,1,34.095448,-118.426087
8,Beverlywood,0.0,1,34.051380,-118.401330
9,Boyle Heights,0.0,1,34.040040,-118.210500
12,Canoga Park,0.0,1,34.202390,-118.601560
13,Carthay,0.0,1,34.176397,-118.222743


In [41]:
cluster1.shape

(68, 5)

Cluster 1 groups together 68 neighborhoods with no existence of fitness centers in proximity. The venue data for these neighborhoods generated 0 occurrences of a fitness center within a 2000 meter radius from their center points.

### Cluster 2 (Blue)

In [42]:
cluster2 = LA_merged.loc[LA_merged['Cluster Labels'] == 2]
cluster2

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
11,Broadway-Manchester,0.030000,2,33.995234,-118.467059
34,Glassell Park,0.026316,2,34.119250,-118.230390
76,Playa del Rey,0.026667,2,33.959250,-118.447990
87,Studio City,0.030000,2,34.144530,-118.395610
98,Venice,0.030000,2,33.987540,-118.472150
113,Woodland Hills,0.030000,2,34.168890,-118.611480


In [43]:
cluster2.shape

(4, 5)

Cluster 2 groups together 4 neighborhoods with a mid-high concentration of fitness centers in proximity. The venue data for these neighborhoods generated 3 occurrences of a fitness center within a 2000 meter radius from their center points.

## Cluster 3 (Green)

In [43]:
cluster3 = LA_merged.loc[LA_merged['Cluster Labels'] == 3]
cluster3

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
3,Atwater Village,0.020000,3,34.119700,-118.258870
6,Beverly Crest,0.023256,3,34.096533,-118.403295
19,Cheviot Hills,0.020000,3,34.034030,-118.410560
21,Cypress Park,0.015625,3,34.094480,-118.226780
22,Del Rey,0.020000,3,33.990820,-118.420620
26,Echo Park,0.020000,3,34.076090,-118.255810
67,North Hollywood,0.020000,3,34.169820,-118.378990
71,Palms,0.020000,3,34.019860,-118.420960
85,Silver Lake,0.020000,3,34.082730,-118.273070
91,Tarzana,0.020000,3,34.175290,-118.550100


In [45]:
cluster3.shape

(9, 5)

Cluster 3 groups together 9 neighborhoods with a low-mid concentration of fitness centers in proximity. The venue data for these neighborhoods generated at least 2 occurrences of a fitness center within a 2000 meter radius from their center points.

### Cluster 4 (Orange)

In [44]:
cluster4 = LA_merged.loc[LA_merged['Cluster Labels'] == 4]
cluster4

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
10,Brentwood,0.041237,4,34.06626,-118.47033
84,Sherman Oaks,0.040000,4,34.15106,-118.44434


In [47]:
cluster4.shape

(2, 5)

Cluster 4 groups together 2 neighborhoods with a high concentration of fitness centers in proximity. The venue data for these neighborhoods generated at least 4 occurrences of a fitness center within a 2000 meter radius from their center points.

<a id="ref10"></a> 
# Discussion

Based on the clustering data over the map of Los Angeles, the majority of neighborhoods under Los Angeles county lack access to nearby fitness centers. Very few neighborhoods harbor a mid to high concentration of local fitness centers present. In fact, clusters 2, 3 and 4, which contain a mid to high concentration of fitness centers, can be combined and they would only total 17 neighborhoods out of the 114. Unsurprisingly, clusters 2, 3, and 4 contain suburban neighborhoods with a higher income population. Judging also by their higher concentration of fitness centers, it is safe to recommend to avoid building new fitness centers in these areas since they already have competition established. Even more so, the intent is to give more local access to neighborhoods that lack fitness centers.

Continuing on, the second largest cluster is cluster 0, which list neighborhoods with at least one occurrence of a fitness center in proximity. Cluster 0 contained 30 neighborhoods out of the 114 in Los Angeles and geographically was concentrated towards Western Los Angeles and North West Los Angeles. Competition for memberships is probably not high in these neighborhoods so they can be considered as a secondary tier of neighborhoods for building more fitness centers around. 

Every neighborhood that did not generate an occurence of a fitness center within a 2000 meter radius from their center point was assigned to Cluster 1. Cluster 1 had a total of 67 out of 114 neighborhoods assigned to its cluster. Geographically, the neighborhoods under cluster 1 that are located in the most southern side of Los Angeles have the least availability to access a fitness center in proximity.This discrepancy could be due to the fact high concentration of low income residents reside in the southern side of Los Angeles which in the past might have deterred property developers from establishing fitness centers there. 

In Addition, there are also several other notable geographic concentrations from cluster 1 that display a lack of available neighborhoods such as East Los Angeles, Central Los Angeles, and the most northern regions of Los Angeles that hug San Fernando. These city regions from cluster 1 are also high population dense areas so there is definitely a high probability of demand for more convenient fitness centers in these areas. 



<a id="ref11"></a> 
# Conclusion

Despite there being some limitations as far as how much venue data can be queried from Foursquare, enough venue data was gathered where an accurate mean frequency of occurrence for gyms and fitness centers can be obtained for each Los Angeles neighborhood. From these mean frequency of occurrences for gyms and fitness centers, five unique clusters were able to be made and each held notable geographic insights. In Particular the most stand out cluster that can be referred to as the ideal list of neighborhoods to target for building new fitness centers is cluster 1. Cluster 1 revealed that there are at least three big and populous regions in Los Angeles that have zero fitness centers established and they are Southern Los Angeles, East los Angeles, and Central Los Angeles. While these regions have historically housed low-income residents, this factor should not limit property developers from pursuing to build fitness centers in these regions. In fact, property developers should aim to build a market of low-cost membership fitness centers in these regions. Better yet, the city of Los Angeles can serve the health needs of its low-income residents better by building community fitness recreational centers in these neighborhoods as well. Overall, there is a market and need to build more fitness centers for the neighborhoods listed under cluster 1. 
